In [315]:
import pandas as pd

In [316]:
data = pd.read_csv("fake_job_postings.csv") #wczytywanie pliku z bazą danych


In [317]:
data.head() #wyświetlenie kilku pierwszych wierszy danych

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [318]:
data.info() #informacje o danych

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   job_id               17880 non-null  int64 
 1   title                17880 non-null  object
 2   location             17534 non-null  object
 3   department           6333 non-null   object
 4   salary_range         2868 non-null   object
 5   company_profile      14572 non-null  object
 6   description          17879 non-null  object
 7   requirements         15184 non-null  object
 8   benefits             10668 non-null  object
 9   telecommuting        17880 non-null  int64 
 10  has_company_logo     17880 non-null  int64 
 11  has_questions        17880 non-null  int64 
 12  employment_type      14409 non-null  object
 13  required_experience  10830 non-null  object
 14  required_education   9775 non-null   object
 15  industry             12977 non-null  object
 16  func

In [319]:
data.describe() #podsumowanie danych

,job_id,telecommuting,has_company_logo,has_questions,fraudulent
count,17880.000000,17880.000000,17880.000000,17880.000000,17880.000000
mean,8940.500000,0.042897,0.795302,0.491723,0.048434
std,5161.655742,0.202631,0.403492,0.499945,0.214688
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,4470.750000,0.000000,1.000000,0.000000,0.000000
50%,8940.500000,0.000000,1.000000,0.000000,0.000000
75%,13410.250000,0.000000,1.000000,1.000000,0.000000
max,17880.000000,1.000000,1.000000,1.000000,1.000000


In [320]:
data.isnull().sum() #podsumowanie wartości n/a w kazdej kolumnie

job_id                     0
title                      0
location                 346
department             11547
salary_range           15012
company_profile         3308
description                1
requirements            2696
benefits                7212
telecommuting              0
has_company_logo           0
has_questions              0
employment_type         3471
required_experience     7050
required_education      8105
industry                4903
function                6455
fraudulent                 0
dtype: int64

In [321]:
data=data.dropna() #usunięcie wierszy z brakującymi wartościami

In [322]:
#przekształcenie na małe litery
data['description'] = data['description'].str.lower()
data['required_education'] = data['required_education'].str.lower()

In [323]:
from sklearn.feature_extraction.text import TfidfVectorizer 

#wektoryzacja dla kolumny description 
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000) #vectorizer który przekształca tekst na wektory liczbowo reprezentujace czestotliwosc wystapien slow
X_text = vectorizer.fit_transform(data['description'])


In [324]:
from sklearn.preprocessing import OneHotEncoder

#konwersja na reprezentacje numeryczna (ograniczona liczba wartości)
encoder = OneHotEncoder() #koduje kategorie na wektory binarne 
X_edu = encoder.fit_transform(data[['required_education']])


In [325]:
from scipy.sparse import hstack

#łączenie cech
X = hstack([X_text, X_edu])
Y = data['fraudulent']

In [326]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3) #podział danych na train i test

In [327]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import cross_val_score 



#tworzenie i trenowanie modelu K - nearest neighbors

model = KNeighborsClassifier()
model.fit(X_train, Y_train)


#przewidywanie na zestawie testowym
Y_pred = model.predict(X_test)

#wyświetlanie dokładności i raportu
print("Accuracy:", accuracy_score(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))

#sprawdzanie powtarzalności walidacja krzyżową
scores = cross_val_score(model, X, Y, cv=5) 
print("Cross-validated scores:", scores)

Accuracy: 0.9613733905579399
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       216
           1       0.79      0.65      0.71        17

    accuracy                           0.96       233
   macro avg       0.88      0.82      0.84       233
weighted avg       0.96      0.96      0.96       233

Cross-validated scores: [0.93548387 0.95483871 0.96129032 0.96129032 0.96103896]
